In [1]:
from google.colab import files
uploaded = files.upload()

Saving Loans.csv to Loans.csv


In [5]:
import io
import pandas as pd
data = pd.read_csv(io.BytesIO(uploaded['Loans.csv']))
print(data.head(2))

   credit.policy             purpose  ...  not.fully.paid  annualincome
0              1  debt_consolidation  ...               1         45000
1              1           all_other  ...               1         60000

[2 rows x 15 columns]


In [12]:
d = data.head()
print(d)

   credit.policy  purpose  int.rate  ...  pub.rec  not.fully.paid  annualincome
0              1        0    0.1496  ...        1               1         45000
1              1        1    0.1114  ...        0               1         60000
2              1        0    0.1343  ...        0               1        145000
3              1        1    0.1059  ...        0               1         33990
4              1        0    0.1501  ...        0               1        213000

[5 rows x 15 columns]


In [6]:
print(data.describe())

       credit.policy     int.rate  ...  not.fully.paid  annualincome
count    5000.000000  5000.000000  ...     5000.000000  5.000000e+03
mean        0.896200     0.120816  ...        0.306600  6.626021e+04
std         0.305031     0.025336  ...        0.461128  5.686419e+04
min         0.000000     0.060000  ...        0.000000  2.000000e+03
25%         1.000000     0.100800  ...        0.000000  3.800000e+04
50%         1.000000     0.121800  ...        0.000000  5.500000e+04
75%         1.000000     0.137900  ...        1.000000  7.900000e+04
max         1.000000     0.216400  ...        1.000000  2.039784e+06

[8 rows x 14 columns]


In [ ]:
print(data[data['not.fully.paid'] == 1].describe())

       credit.policy     int.rate  installment  log.annual.inc          dti  \
count    1533.000000  1533.000000  1533.000000     1533.000000  1533.000000   
mean        0.661448     0.132452   342.785114       10.885023    13.195838   
std         0.473372     0.025495   223.948527        0.666718     7.006769   
min         0.000000     0.070500    15.910000        7.600902     0.000000   
25%         0.000000     0.115400   168.640000       10.491274     7.830000   
50%         1.000000     0.131600   287.310000       10.878047    13.340000   
75%         1.000000     0.148200   491.300000       11.276633    18.830000   
max         1.000000     0.216400   926.830000       13.458836    29.960000   

              fico  days.with.cr.line     revol.bal   revol.util  \
count  1533.000000        1533.000000  1.533000e+03  1533.000000   
mean    697.828441        4393.541259  2.106629e+04    52.255075   
std      33.756808        2431.785491  4.990569e+04    29.057906   
min     617.0000

In [ ]:
print(data[data['not.fully.paid'] == 0].describe())

       credit.policy     int.rate  installment  log.annual.inc          dti  \
count         3467.0  3467.000000  3467.000000     3467.000000  3467.000000   
mean             1.0     0.115671   293.089201       10.923667    11.916432   
std              0.0     0.023498   182.272593        0.566024     6.603058   
min              1.0     0.060000    15.690000        8.342840     0.000000   
25%              1.0     0.096300   159.920000       10.585573     6.775000   
50%              1.0     0.116600   249.680000       10.915088    11.860000   
75%              1.0     0.131600   394.360000       11.277203    17.120000   
max              1.0     0.208600   914.420000       14.528354    29.420000   

              fico  days.with.cr.line      revol.bal   revol.util  \
count  3467.000000        3467.000000    3467.000000  3467.000000   
mean    716.717335        4562.523339   13576.013268    43.804753   
std      36.935882        2411.216297   16685.502884    28.800678   
min     627.

In [7]:
print("Number of loans that have 50% credit utilization and defaulted: ")
print(len(data[(data['revol.util'] > 50.00) & (data['not.fully.paid'] == 1)]))
print("Number of loans that have 50% credit utilization and not defaulted: ")
print(len(data[(data['revol.util'] > 50.00) & (data['not.fully.paid'] == 0)]))

Number of loans that have 50% credit utilization and defaulted: 
837
Number of loans that have 50% credit utilization and not defaulted: 
1433


In [8]:
from sklearn import preprocessing
from scipy.stats import boxcox
from sklearn.preprocessing import OneHotEncoder

#Print unique values of purpose
print(pd.Series.unique(data['purpose']))
# Convert purpose to category
data['purpose'] = data['purpose'].astype('category')
#OneHotEcoding
data['purpose'] = pd.get_dummies(data['purpose'], prefix=['purpose'])

#extract dependent variable as label
Y = data['not.fully.paid']
#Drop dependent variable and categorical variable
X = data.drop('not.fully.paid', 1)

#scale dependent variable
X = preprocessing.scale(X)

#print first row of X
print(X[0])
print(X[0].mean(axis=0))
print(X[0].std(axis=0))

['debt_consolidation' 'all_other' 'credit_card' 'small_business'
 'home_improvement' 'educational' 'major_purchase']
[ 0.34032684 -0.57026784  1.13619619 -0.57938823 -0.32964068 -1.23021742
 -1.18644994 -0.55024824 -0.38676608  1.04354413 -0.70711203 -0.32478726
  3.62321686 -0.37391434]
-0.006822001516728127
1.2104049514994222


In [10]:
#Shuffle and split data into 70% in training and 30% in testing
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
print("X_train size = {0}".format(len(X_train)))
print("Y_train size = {0}".format(len(Y_train)))
print("X_test size = {0}".format(len(X_test)))

X_train size = 3500
Y_train size = 3500
X_test size = 1500


In [11]:
#Using Artificial Neural Network
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
import time
from sklearn import metrics
from sklearn.metrics import confusion_matrix

#Fit ANN model to data
startTrain = time.time()
classifier = Sequential()
classifier.add(Dense(activation="relu", kernel_initializer="uniform", units= 6, input_dim=14))
classifier.add(Dropout(rate=0.1)) 
classifier.add(Dense(activation="relu", kernel_initializer="uniform", units=6)) 
classifier.add(Dense(activation="sigmoid", kernel_initializer="uniform", units=1))
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
classifier.fit(np.array(X_train), np.array(Y_train), batch_size=10, epochs=100)
endTrain = time.time()

print("ANN training time (secs): {0}".format(endTrain - startTrain))

#Predict using ANN
startTest = time.time()
Y_pred = classifier.predict_classes(np.array(X_test))
endTest = time.time()

print("ANN prediction time (secs): {0}".format(endTest - startTest))
print("ANN accuracy in testing set: {0}".format(metrics.accuracy_score(Y_test, Y_pred)))
print("ANN F1 score in testing set: {0}".format(metrics.f1_score(Y_test, Y_pred, average='micro')))

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, Y_pred)

print("ANN Confusion Matrix: {0}".format(cm))

Epoch 1/100
350/350 [==============================] - 1s 1ms/step - loss: 0.6475 - accuracy: 0.6901
Epoch 2/100
350/350 [==============================] - 0s 1ms/step - loss: 0.4930 - accuracy: 0.8057
Epoch 3/100
350/350 [==============================] - 0s 1ms/step - loss: 0.4358 - accuracy: 0.8125
Epoch 4/100
350/350 [==============================] - 0s 1ms/step - loss: 0.4392 - accuracy: 0.8131
Epoch 5/100
350/350 [==============================] - 0s 1ms/step - loss: 0.4471 - accuracy: 0.8070
Epoch 6/100
350/350 [==============================] - 0s 1ms/step - loss: 0.4652 - accuracy: 0.7876
Epoch 7/100
350/350 [==============================] - 0s 1ms/step - loss: 0.4316 - accuracy: 0.8131
Epoch 8/100
350/350 [==============================] - 0s 1ms/step - loss: 0.4402 - accuracy: 0.8065
Epoch 9/100
350/350 [==============================] - 0s 1ms/step - loss: 0.4409 - accuracy: 0.8026
Epoch 10/100
350/350 [==============================] - 0s 1ms/step - loss: 0.4478 - accura

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
